In [1]:
import pandas as pd

In [2]:
import tweepy,twitter

In [3]:
auth = tweepy.OAuthHandler('D4cA5heqwzCJryH1GnUzEDopn', 'wjSCGp8t4uP3M1egnzy8lhmja3JR04Pw2B2D01gZHz0Kf6iSvt')
auth.set_access_token('823928942084005889-LCht7E7SJkIya0CBWtiCrbepRrG1KuU', 'wIllRVmp2X2qEra4UYiWGYG5wpWVZo05acQvJ6sf28Iz0')

api = tweepy.API(auth)

In [4]:
suicidal_tweets=pd.read_csv("suicidal_data.csv")

In [5]:
import pickle
import re
import numpy as np
from tqdm import tqdm
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor


In [7]:
def preprocess_tweet(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text+' '.join(emoticons).replace('-', '') 
    return text

In [8]:
tqdm.pandas()
suicidal_tweets['tweet'] = suicidal_tweets['tweet'].progress_apply(preprocess_tweet)

C:\Users\user\Desktop\sample_project\env\lib\site-packages\tqdm\std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|███████████████████████████████████████████████████████████████████████████| 9119/9119 [00:00<00:00, 15287.35it/s]


In [9]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [10]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [11]:
[w for w in tokenizer_porter('a swimmer likes swimming and swims a lot') if w not in stop]

['swimmer', 'like', 'swim', 'swim', 'lot']

In [12]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\(|D|P)',text.lower())
    text = re.sub('[\W]+', ' ', text.lower())
    text += ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in tokenizer_porter(text) if w not in stop]
    return tokenized

## Using the Hashing Vectorizer

In [13]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, 
                         preprocessor=None,tokenizer=tokenizer)

## Building the ml model

In [14]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', random_state=1)

In [15]:
X = suicidal_tweets["tweet"].to_list()
y = suicidal_tweets['label']

## Training the model

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [17]:
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

In [18]:
classes = np.array([0, 1])
clf.partial_fit(X_train, y_train,classes=classes)

SGDClassifier(loss='log', random_state=1)

In [19]:
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.912


In [20]:
clf = clf.partial_fit(X_test, y_test)

## Testing 

In [21]:
label = {0:'negative', 1:'positive'}

In [22]:
phrases = ["suicide","suicidal", "kill myself", "my suicide note", "my suicide letter", "end my life", "never wake up", "can't go on"," cannot go on", "not worth living", "ready to jump", "swleep forever", "want to die", "be dead", "better off without me", "better off dead", "don't want to being here", "tired of living", "die alone", "go to sleep forever", "wanna die", "wanna suicide", "commit suicide", "slit my wrist","cut my wrist", "slash my wrist", "do not want to be here", "want it to be over", "want to be dead", "nothing to live for", "not worth living","ready to die", "thoughts of suicide", "thoughts of killing myself", "why should i live", "take my own life", "depressed"]

In [23]:
streamed_tweets=[]
for query in phrases:    
    tweets = api.search(q= query + " -filter:retweets", lang="en", show_user=True, result_type="recent")
    for i in tweets:
        print(i.text)
        streamed_tweets.append({'tweet_id': i.id,'tweet_text': i.text, 'user_name': i.user.screen_name})

@EssexPR Lost a dear friend due to suicide, loneliness might just tip some people over the already very thin line,… https://t.co/NHy8MmomrH
Gonna become a reclusive and mysterious French detective with a deep devotion to God that ultimately makes him ques… https://t.co/Y6tvdcH5wE
tw suicide

probably kill myself bc my exams will be over and i'll fail and i have literally no reason to live on i… https://t.co/pjHqqOm7FG
Buying time?? A true impossibility.  Within the now lays infinite grains. cries of more time, quickened our flow, o… https://t.co/a6QKA7Etiz
@LimaVibin Vou ficar com as tuas contas, I'm gonna make it look like a suicide :)
[Suicide] If ya left, I'd miss seein' yer smilin' face, yanno... Promise ya'll show it to me tomorrow?
@Bjorn89er @videogamedeals The same reason why I'm thinking about buying this shirt. 

It's more of a smartass joke… https://t.co/oT7fvjivkR
Veterans suffer the ravages of PTSD  and have an elevated suicide rate, but these self aggrandizing warmongers 

In [24]:
streamed_tweet_df=pd.DataFrame.from_dict(streamed_tweets)
streamed_tweet_df

,tweet_id,tweet_text,user_name
0,1330298556998184961,"@EssexPR Lost a dear friend due to suicide, lo...",manmanelende
1,1330298544411062272,Gonna become a reclusive and mysterious French...,thjnerozzi
2,1330298539369500674,tw suicide\n\nprobably kill myself bc my exams...,YUWIN0
3,1330298520277016583,Buying time?? A true impossibility. Within th...,ChaoticPeace6
4,1330298519861780481,"@LimaVibin Vou ficar com as tuas contas, I'm g...",RubenMont11
...,...,...,...
528,1330298464790581255,Vibing to music and crying on rainy nights hit...,depressed_hoe69
529,1330298451347828744,"i am DEPRESSED, all i want is a doggo of my o...",dualwieldings
530,1330298442619424782,I heard a joke once: Man goes to doctor. Says ...,3mryasser
531,1330298423552118791,jus finished this mcdonald’s n i’m so fucking ...,jadanowada


In [25]:
analysis= pd.DataFrame(columns={"Prediction","Probability"})
analysis["Prediction"].astype(object)
analysis["Probability"].astype(float)

Series([], Name: Probability, dtype: float64)

In [26]:
for row in range(0,len(streamed_tweet_df)):
    example=[streamed_tweet_df.iloc[row,1]]
    X = vect.transform(example)
    print('Prediction: %s   Probability: %.2f%%'
    %(label[clf.predict(X)[0]],np.max(clf.predict_proba(X))*100))
    analysis=analysis.append({'Prediction': label[clf.predict(X)[0]],
                                             'Probability': np.max(clf.predict_proba(X))*100},
                                            ignore_index=True)

Prediction: positive   Probability: 54.56%
Prediction: negative   Probability: 92.72%
Prediction: negative   Probability: 57.42%
Prediction: negative   Probability: 92.43%
Prediction: negative   Probability: 64.60%
Prediction: negative   Probability: 81.07%
Prediction: negative   Probability: 73.34%
Prediction: negative   Probability: 58.92%
Prediction: negative   Probability: 55.32%
Prediction: positive   Probability: 86.16%
Prediction: positive   Probability: 85.23%
Prediction: positive   Probability: 83.93%
Prediction: negative   Probability: 65.92%
Prediction: negative   Probability: 92.09%
Prediction: positive   Probability: 58.65%
Prediction: negative   Probability: 58.18%
Prediction: positive   Probability: 52.96%
Prediction: positive   Probability: 95.73%
Prediction: negative   Probability: 66.51%
Prediction: positive   Probability: 85.25%
Prediction: negative   Probability: 70.18%
Prediction: positive   Probability: 62.37%
Prediction: negative   Probability: 95.83%
Prediction:

In [27]:
merge=streamed_tweet_df.merge(analysis, how="inner", left_index=True, right_index=True)

In [28]:
merge

,tweet_id,tweet_text,user_name,Prediction,Probability
0,1330298556998184961,"@EssexPR Lost a dear friend due to suicide, lo...",manmanelende,positive,54.563773
1,1330298544411062272,Gonna become a reclusive and mysterious French...,thjnerozzi,negative,92.715919
2,1330298539369500674,tw suicide\n\nprobably kill myself bc my exams...,YUWIN0,negative,57.419148
3,1330298520277016583,Buying time?? A true impossibility. Within th...,ChaoticPeace6,negative,92.434784
4,1330298519861780481,"@LimaVibin Vou ficar com as tuas contas, I'm g...",RubenMont11,negative,64.600993
...,...,...,...,...,...
528,1330298464790581255,Vibing to music and crying on rainy nights hit...,depressed_hoe69,negative,93.523375
529,1330298451347828744,"i am DEPRESSED, all i want is a doggo of my o...",dualwieldings,negative,84.971472
530,1330298442619424782,I heard a joke once: Man goes to doctor. Says ...,3mryasser,positive,73.824992
531,1330298423552118791,jus finished this mcdonald’s n i’m so fucking ...,jadanowada,positive,60.319875


In [29]:
mask = (merge['Probability'].ge(80)) & (merge['Prediction']=='positive')
merge['mask'] = np.where(mask,'Flagged', 'no problem')
merge

,tweet_id,tweet_text,user_name,Prediction,Probability,mask
0,1330298556998184961,"@EssexPR Lost a dear friend due to suicide, lo...",manmanelende,positive,54.563773,no problem
1,1330298544411062272,Gonna become a reclusive and mysterious French...,thjnerozzi,negative,92.715919,no problem
2,1330298539369500674,tw suicide\n\nprobably kill myself bc my exams...,YUWIN0,negative,57.419148,no problem
3,1330298520277016583,Buying time?? A true impossibility. Within th...,ChaoticPeace6,negative,92.434784,no problem
4,1330298519861780481,"@LimaVibin Vou ficar com as tuas contas, I'm g...",RubenMont11,negative,64.600993,no problem
...,...,...,...,...,...,...
528,1330298464790581255,Vibing to music and crying on rainy nights hit...,depressed_hoe69,negative,93.523375,no problem
529,1330298451347828744,"i am DEPRESSED, all i want is a doggo of my o...",dualwieldings,negative,84.971472,no problem
530,1330298442619424782,I heard a joke once: Man goes to doctor. Says ...,3mryasser,positive,73.824992,no problem
531,1330298423552118791,jus finished this mcdonald’s n i’m so fucking ...,jadanowada,positive,60.319875,no problem


In [34]:
for row in range(len(merge)):
    if merge['mask'][row]=='Flagged':
        print(row, merge['user_name'][row], merge['tweet_text'][row])

9 GlideChamp 🔫 tw: suicide etc  
small reminder..

Please don't talk about suicide, Self hurting , and something similar like th… https://t.co/KBcLUICkob
10 lxlwi Icaaaaaannnnnnnn't i commit suicide
11 ataratic_ a twizzler a day keeps the suicide away 🤪
17 brattycrybaby I'm so fucking suicidal. Literally someone tell me how to end it. I can't.
19 ANTMlX corey posting suicidal shit on the tl again
49 F8webb Every time an assignment gets posted my suicide note does get longer
52 JbShibe @prodegyxtinct @gawrgura I'm mentioning this in my suicide note
56 Dyingathreeam btw this is my suicide note
61 dongjunstan tw // suicide

they think that i like being here when she saw my fucking suicide letter
68 diptychbitch in the first suicide letter I ever wrote when I was 8 years old I list one of my reasons as the desire to meet Theo… https://t.co/SSOr5qzF4y
196 hairywonwoo you are all better off without me, i know that. ive been on here for nearly a whole year now, and ive never liked b… https://